### Downloading the necessary file/documents

In [1]:
!rm documents.json
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-08-19 16:02:11--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.006s  

2024-08-19 16:02:11 (106 MB/s) - ‘documents.json’ saved [658332/658332]



### Importing the necessary libraries

In [2]:
import json # built-in library
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm import tqdm

### Loading Data for Knowledge Base


In [3]:
# loading the JSON raw file

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
# converting our JSON file into a list object

documents = [] 

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
# lets sample the output
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Implementing ElasticSearch

In [6]:
# connecting to the es client and defining our index

es_client = Elasticsearch("http://localhost:9200") 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# lets delete and create a new index if it exists for ease of re-runs
if es_client.indices.exists(index="course-questions"):
    es_client.indices.delete(index="course-questions")

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
# Check the health of the cluster to see if the changes have taken effect

cluster_health = es_client.cluster.health(index=index_name)

print(cluster_health)

{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [8]:
# now to populate our index with the data from our JSON file

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:24<00:00, 39.37it/s]


In [9]:
# now to define the elasticsearch function for querying - as part of the RAG framework

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

### Generating responses from Ollama models using OpenAI framework

In [10]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [11]:
# funtion to build the prompt

def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip() # so that there are no extra line breaks

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [12]:
# now function for the LLM - model is gemma2

def llm(prompt):

    response = client.chat.completions.create(
        model='gemma2:2b',
        messages=[{'role':'user',
                   'content': prompt}]
    )

    return response.choices[0].message.content

In [13]:
# so now bringing them all together - the RAG flow function
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. \n"

In [15]:
print(rag('How do I run Kafka?'))

To run Kafka, in the project directory, run: 
`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java` 

